<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- KT4 파일로 500명 샘플링해서 사용자 로그 quit 횟수, timestamp 차이(소요시간), mobile/web 비율, submit 비율 계산함
- 500명 행동 패턴 통계 출력

In [ ]:
import zipfile, pandas as pd, random, numpy as np
import matplotlib.pyplot as plt

# ============================================
# 1. ZIP 내부에서 직접 파일 접근 (압축 해제 X)
# ============================================
zip_path = "/content/EdNet-KT4.zip"
zf = zipfile.ZipFile(zip_path)
file_list = [f for f in zf.namelist() if f.endswith(".csv")]
print("총 사용자 수:", len(file_list))

# 500명 샘플링
sample_files = random.sample(file_list, 500)
user_stats = []

# ============================================
# 2. 사용자별 통계 계산
# ============================================
for f in sample_files:
    try:
        with zf.open(f) as fp:
            df = pd.read_csv(fp)
            if df.empty:
                continue

            user_id = f.split("/")[-1].replace(".csv", "")
            df = df.sort_values("timestamp")

            # --- quit 횟수 ---
            quit_count = (df["action_type"] == "quit").sum()

            # --- 소요시간 ---
            duration = df["timestamp"].iloc[-1] - df["timestamp"].iloc[0]

            # --- 플랫폼 비율 ---
            mobile_ratio = (df["platform"] == "mobile").mean()
            web_ratio = (df["platform"] == "web").mean()

            # --- 제출 비율 (submit / 문제 수) ---
            submit_count = (df["action_type"] == "submit").sum()
            unique_q = df[df["item_id"].astype(str).str.startswith("q")]["item_id"].nunique()
            submit_ratio = submit_count / unique_q if unique_q > 0 else np.nan

            user_stats.append({
                "user_id": user_id,
                "quit_count": quit_count,
                "duration_ms": duration,
                "mobile_ratio": mobile_ratio,
                "web_ratio": web_ratio,
                "submit_ratio": submit_ratio
            })
    except Exception as e:
        print(f"{f} 오류: {e}")

# ============================================
# 3. 요약 결과 정리
# ============================================
stats_df = pd.DataFrame(user_stats)
print("\n✅ 사용자별 통계 미리보기:")
print(stats_df.head())

print("\n📈 전체 요약 통계:")
print(stats_df.describe())

# ============================================
# 4. 시각화
# ============================================
plt.figure(figsize=(8,4))
stats_df["quit_count"].hist(bins=30, color="lightcoral")
plt.title("Quit 행동 횟수 분포")
plt.xlabel("quit_count")
plt.ylabel("사용자 수")
plt.show()

plt.figure(figsize=(8,4))
stats_df["submit_ratio"].hist(bins=30, color="mediumseagreen")
plt.title("문항 수 대비 Submit 비율 분포")
plt.xlabel("submit_ratio")
plt.ylabel("사용자 수")
plt.show()

plt.figure(figsize=(6,4))
stats_df[["mobile_ratio","web_ratio"]].mean().plot(kind='bar', color=["orange","skyblue"])
plt.title("평균 플랫폼 비율 (Mobile vs Web)")
plt.ylabel("비율")
plt.show()

plt.figure(figsize=(8,4))
plt.scatter(stats_df["duration_ms"]/60000, stats_df["quit_count"], alpha=0.6, c="slateblue")
plt.title("사용자별 소요시간 vs Quit 횟수")
plt.xlabel("소요시간(분)")
plt.ylabel("Quit 횟수")
plt.show()

print("\n✅ 시각화 완료! 데이터프레임 변수명: stats_df")
